In [13]:
import numpy as np
import random
import matplotlib.pyplot as plt
import pandas as pd
import sys,csv,os,IPython,string,re
from numpy import NaN as NA
pd.options.display.max_rows=10
pd.options.display.max_columns=20

# 8.1 Hierarchical Indexing
###### Help you to work with higher dimensional data in a lower dimensional form

In [14]:
# create a series with the multiindex that have 2 level
shop1=pd.Series(data=np.arange(10),
                index=[['A','A','B','B','C','C','C','D','D','D'],
                       ['apple','ana','banana','boom','cut','cd','CV','df','df0','df1']],
                name='shop1')
# create a df from the series 'shop1'
df=pd.DataFrame(data=shop1)
# put the name for index at each level
df.index.rename(names='Alphabet',level=0,inplace=True)
df.index.rename(names='Name',level=1,inplace=True)
# create more a series with the index=shop1.index
shop2=pd.Series(data=np.arange(10,0,-1),index=shop1.index,name='shop2')
# join df with series'shop2'
df=df.join(other=shop2)
# choose a subset of df from the [index(level=0)][index(level=1)] [[columns]]
subset=df['A':'C'] [1:6] [['shop1','shop2']] # indices
# choose [index(level=0)],[col]
subset1=df.loc[['A','B','C'],['shop1','shop2']]
# unstack the multiiindex to 1-dimensional index
unstack_0=df.unstack(level=0,fill_value=0)# unstack level=0 corresponding with df.index=level(1)
unstack_1=df.unstack(level=1,fill_value=0)
# stack the unstack_0 of df
stack=unstack_0.stack(level=1,dropna=True)# similar to the df
# convert from 1-d Columns to Multi-dimensional columns by pd.MultiIndex.from_array
df.columns=pd.MultiIndex.from_arrays(arrays=[['Quantile','Quantile'],['Shop1','Shop2']],names=['Indicator','Of Shop'])
df

Indicator       Quantile      
Of Shop            Shop1 Shop2
Alphabet Name                 
A        apple         0    10
         ana           1     9
B        banana        2     8
         boom          3     7
C        cut           4     6
         cd            5     5
         CV            6     4
D        df            7     3
         df0           8     2
         df1           9     1

#### Reordering and Sorting Levels 

In [15]:
# reordered(~swap) level of multiindex or columns
df.swaplevel(axis=0)# automatically swap level if multiindex only have 2 level
# reordered axis (0,1) to (1,0)
df.swapaxes(axis1=1,axis2=0)
# sort by the name of index of certain level
df.sort_index(axis=0,level=1,ascending=False)
# combine swap level and sort for a certain level and index
df.swaplevel(axis=0).sort_index(axis=0,ascending=False)

Indicator       Quantile      
Of Shop            Shop1 Shop2
Name   Alphabet               
df1    D               9     1
df0    D               8     2
df     D               7     3
cut    C               4     6
cd     C               5     5
boom   B               3     7
banana B               2     8
apple  A               0    10
ana    A               1     9
CV     C               6     4

### Summary Statistics by Level

In [16]:
df.sum(axis=1,min_count=True,skipna=True,numeric_only=True)
df.mean(axis=1,level=0,skipna=True,numeric_only=True)
df.std(axis=0,level=0,skipna=True,numeric_only=True)

Indicator  Quantile          
Of Shop       Shop1     Shop2
Alphabet                     
A          0.707107  0.707107
B          0.707107  0.707107
C          1.000000  1.000000
D          1.000000  1.000000

### Indexing with a DF's columns 

In [17]:
data=pd.DataFrame({'a':range(7),'b':range(7,0,-1),
                  'c':['one','one','one','two','two','two','two'],
                  'd':[0,1,2,0,1,2,3]})
data.set_index(keys=['c','d'],drop=False,inplace=True,append=False)
data
data.reset_index(drop=True,level=[0,1],col_fill=['c','d'])

,a,b,c,d
0,0,7,one,0
1,1,6,one,1
2,2,5,one,2
3,3,4,two,0
4,4,3,two,1
5,5,2,two,2
6,6,1,two,3


# 8.2 Combining and Merging Datasets
### DB-Style DF joins

In [18]:
# MANY-TO-ONE
df1=pd.DataFrame({'key':['b','b','a','c','a','a','b'],
                 'data1':range(7)})
df2=pd.DataFrame({'key':['a','b','d'],
                 'data2':range(3)})
# Merge the left type from 2df with the primary is 'key'
pd.merge(df1,df2,how='left',on=['key'])
# change the name of columns and user (left_on,right_on) for corresponding df
df1.columns=['key1','data1']
df2.columns=['key2','data2']
pd.merge(right=df2,left=df1,how='inner',left_on='key1',right_on='key2')
#-----------------------------------------------------------------------
# MANY-TO-MANY
df1 = pd.DataFrame({'key1': ['b', 'b', 'a', 'c', 'a', 'b'],
                    'data1': range(6)})
df2 = pd.DataFrame({'key2': ['a', 'b', 'a', 'b', 'd'],
                    'data2': range(5)})
# Cartesian product from the df1,df2 with the element that appeared in 2 df
pd.merge(df1,df2,left_on='key1',right_on='key2',how='outer')
#--------------------------------------------------------------------
# Merge with many 'keys' as['key1','key2']
left=pd.DataFrame({'key1':['foo','foo','bar2'],
                  'key2':['one','two','one'],
                  'lval':[1,2,3]})
right=pd.DataFrame({'key1':['foo','foo','bar','bar1'],
                  'key2':['one','one','one','two'],
                  'rval':[4,5,6,7]})
pd.merge(left,right,how='outer',on=['key1','key2'])
# suffixes can be used to add suffixes on the same name of columns in both of 2 df
# indicator col that named 'source' to means that the source of joined df
pd.merge(left,right,how='outer',on='key1',suffixes=('_left','_right'),indicator='source')

,key1,key2_left,lval,key2_right,rval,source
0,foo,one,1.0,one,4.0,both
1,foo,one,1.0,one,5.0,both
2,foo,two,2.0,one,4.0,both
3,foo,two,2.0,one,5.0,both
4,bar2,one,3.0,NaN,NaN,left_only
5,bar,NaN,NaN,one,6.0,right_only
6,bar1,NaN,NaN,two,7.0,right_only


### Merging on Index 

In [19]:
left1=pd.DataFrame({'key':['a','b','a','a','b','c'],'value':range(6)})
right1=pd.DataFrame({'group_val':[3.5,7]},index=['a','b'])
# merge 2 index to 1 combining index
pd.merge(left1,right1,how='outer',left_index=True,right_index=True)
# merge 1 index with 1 key
merge_index_key=pd.merge(left1,right1,how='outer',left_on='key',right_index=True,indicator='source')
merge_index_key.sort_index(ascending=True)
# merge 2 df by 2 index by join
left2=pd.DataFrame([[1.,2.],[3.,4.],[5.,6.]],index=['a','c','e'],columns=['Ohio','Nevada'])
right2=pd.DataFrame([[7.,8.],[9.,10.],[11.,12.],[13,14]],index=['b','c','d','e'],columns=['Missouri','Alabama'])
# join 2 df by index
left2.join(right2,how='outer')
# join with more 1 df
more=pd.DataFrame([[3,4],[5,6],[10,-1]],index=['a','b','c'],columns=['Hanoi','HaiPhong'])
left2.join([right2,more],how='outer',sort=True)

C:\Users\Ngoc\Anaconda3\lib\site-packages\pandas\core\frame.py:6359: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  verify_integrity=True)


,Ohio,Nevada,Missouri,Alabama,Hanoi,HaiPhong
a,1.0,2.0,NaN,NaN,3.0,4.0
b,NaN,NaN,7.0,8.0,5.0,6.0
c,3.0,4.0,9.0,10.0,10.0,-1.0
d,NaN,NaN,11.0,12.0,NaN,NaN
e,5.0,6.0,13.0,14.0,NaN,NaN


### Concatenating Along an Axis

In [20]:
# create a df (4,2)
df=pd.DataFrame([[random.randint(1,10),random.randint(1,100)]for x in range(4)],index=list('abcd'),columns=['A','B'])
print(df)
# concatenated by axis=1
pd.concat(objs=[df*2,df['A']+df['B']],axis=1)
# concatenate and categories 2 class by keys['AA','BB']
df2=pd.concat(objs=[df,df['A']*2+df['B']],axis=0,keys=['AA','BB'],join='outer')
df2.unstack()
# concatenate with 'keys' to categories and names for 'level_0,_1'
data1=pd.DataFrame({'one':[12,13],'two':[10,20]},index=['a','b'])
data2=pd.DataFrame({'three':[21,31,100],'four':[110,220,40]},index=['a','b','c'])
data3=pd.concat(objs=[data1,data2],keys=['lvl1','lvl2'],axis=1,sort=True,names=['level_0','level_1'])
# concatenate and categories for multiindex
pd.concat(objs=[data3,data3*0.5],axis=0,keys=['A','B'],names=['Alphabet','Sample'])
# ignore_index then concatenated by index
pd.concat(objs=[data3,data3*2],axis=0,ignore_index=True)

   A   B
a  8  89
b  1  47
c  4  25
d  7  13


level_0  lvl1        lvl2     
level_1   one   two three four
0        12.0  10.0    21  110
1        13.0  20.0    31  220
2         NaN   NaN   100   40
3        24.0  20.0    42  220
4        26.0  40.0    62  440
5         NaN   NaN   200   80

### Combining Data with Overlap

In [21]:
a = pd.Series([np.nan, 2.5, np.nan, 3.5, 4.5, np.nan],index=['f', 'e', 'd', 'c', 'b', 'a'])
b = pd.Series(np.arange(len(a), dtype=np.float64),index=['f', 'e', 'd', 'c', 'b', 'a'])
a.combine_first(other=b)# having value~ (np.where(pd.isnull(a),b,a))
data2.columns=['one','two']
# data1 combine with data2 but data1 overwritten data2 if this position has existed 2 values
data1.combine_first(data2)

,one,two
a,12.0,10.0
b,13.0,20.0
c,100.0,40.0


# 8.3 Reshaping and Pivoting

In [22]:
data=pd.DataFrame(np.arange(6).reshape((2,3)),index=pd.Index(['Ohio','Colorado'],name='state'),
                 columns=pd.Index(['One','Two','Three'],name='number'))
print(data)
# stack the df from 1-d index row to multiindex
stack=data.stack()
# unstack the multiindex to 
unstack=stack.unstack(level=0) # ~ stack.unstack(level='state')
# show the each value NA in each df that formed the combined df
ab=pd.concat(objs=[a,b],axis=0,keys=['A','B'])
ab.unstack()
ab.unstack().stack(dropna=False)# ~pd.concat without dropNA
# transpose the pivot of df with unstack().stack()
shop=pd.DataFrame({'shop1':stack,'shop2':stack*2},columns=pd.Index(['shop1','shop2'],name='SHOP'))
shop.unstack(0).stack('SHOP')

number    One  Two  Three
state                    
Ohio        0    1      2
Colorado    3    4      5


state         Colorado  Ohio
number SHOP                 
One    shop1         3     0
       shop2         6     0
Two    shop1         4     1
       shop2         8     2
Three  shop1         5     2
       shop2        10     4

### Pivoting 'Long' to 'Wide' Format 

In [64]:
raw=pd.read_csv('Pydata-book\\pydata-book-2nd-edition\\examples\\macrodata.csv')
date=pd.PeriodIndex(year=raw.year,quarter=raw.quarter,name='date')
columns=pd.Index(['realgdp','infl','unemp'],name='item')
# set the columns with value for the data from 'raw'
data=raw.reindex(columns=columns)
# set indes as the date.to_timestamp
times_index_data=data.set_index(keys=date.to_timestamp(how='end'))
# convert from date to date_df
date_df=pd.DataFrame(date.to_timestamp(how='end',freq='D'))
# add the date_df to data as index of data
data.index=date_df['date'] # data~ times_index_data
# convert data to multiindex with date(~lvl1) and item(~lvl2)
multiindex_data=pd.DataFrame(data.stack('item'))
multiindex_data.unstack(1)# similar to data
# give back the index to the sequence of number by reset_index
reset_index=multiindex_data.reset_index() # the column that contains all value is named '0'
# rename the columns'0' contains all value
long_df=reset_index.rename(columns={0:'value'},inplace=False)# long-format for multiple time series
# way to back to format 'data' by mannually
back_data=long_df.set_index(keys=['date','item']).unstack('item')
#-------------------------------------------------------------
# use pivot table to design the index,col,value to back the data
long_df.pivot(index='date',columns='item',values='value')# similar 'data'
long_df.pivot(index='date',columns='item')# hierarchical columns 
pd.DataFrame(long_df.set_index(['date','item']).unstack('item'))# similar to the previous code

value                 
item        infl    realgdp unemp
date                             
1959-03-31  0.00   2710.349   5.8
1959-06-30  2.34   2778.801   5.1
1959-09-30  2.74   2775.488   5.3
1959-12-31  0.27   2785.204   5.6
1960-03-31  2.31   2847.699   5.2
...          ...        ...   ...
2008-09-30 -3.16  13324.600   6.0
2008-12-31 -8.79  13141.920   6.9
2009-03-31  0.94  12925.410   8.1
2009-06-30  3.37  12901.504   9.2
2009-09-30  3.56  12990.341   9.6

[203 rows x 3 columns]

### Pivoting 'Wide' to 'Long' Format

In [74]:
df=pd.DataFrame({'key':['foo','bar','baz'],
                'A':[1,2,3],
                'B':[4,5,6],
                'C':[7,8,9]})
print(df)
# melt for form the long-format
melted=df.melt(id_vars=['key','A'],var_name='Var',value_name='Value')
# pivot only use for 1-d index
melted.pivot(index='key',columns='Var',values='Value')
# pivot_table use for multiindex df
melted.pivot_table(index=['key','A'],columns='Var',values='Value')


   key  A  B  C
0  foo  1  4  7
1  bar  2  5  8
2  baz  3  6  9


,Var,B,C
key,A,,
bar,2,5,8
baz,3,6,9
foo,1,4,7
